# [Pyomo.GDP](./index.ipynb) Logical Expression System Demo - Eight Process Problem (8PP)

This is a reproduction of the eight process problem, example 3 in:

> *Select Optimal Process From Within Given Superstructure*.\
Marco Duran, PhD Thesis, 1984.\
Carnegie Mellon University, Pittsburgh, PA.

This code relies on the logic-v1 branch at https://github.com/qtothec/pyomo/tree/logic-v1

In [1]:
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.core.expr.logical_expr import *
from pyomo.core.plugins.transform.logical_to_linear import update_boolean_vars_from_binary

In [2]:
m = ConcreteModel()
m.streams = RangeSet(2, 25, doc="process streams")
m.units = RangeSet(1, 8, doc="process units")
m.fixed_cost = Param(m.units, doc="fixed cost", initialize={
    1: 5, 2: 8, 3: 6, 4: 10, 5: 6, 6: 7, 7: 4, 8: 5
})
m.unit_fixed_cost = Var(m.units, initialize=0, bounds=(0, 10))
for unit, cost in m.fixed_cost.items():
    m.unit_fixed_cost[unit].setub(cost)
stream_variable_cost_data = {
    3: -10, 5: -15, 9: -40, 19: 25, 21: 35, 25: -35,
    17: 80, 14: 15, 10: 15, 2: 1, 4: 1, 18: -65, 20: -60,
    22: -80}
m.stream_variable_cost = Param(m.streams, initialize=stream_variable_cost_data, default=0)

stream_flow_initial_guess = {
    2: 2, 3: 1.5, 6: 0.75, 7: 0.5, 8: 0.5, 9: 0.75, 11: 1.5,
    12: 1.34, 13: 2, 14: 2.5, 17: 2, 18: 0.75, 19: 2, 20: 1.5,
    23: 1.7, 24: 1.5, 25: 0.5}
stream_flow_upper_bound_data = {3: 2, 5: 2, 9: 2, 10: 1, 14: 1, 17: 2, 19: 2, 21: 2, 25: 3}

m.flow = Var(m.streams, domain=NonNegativeReals, bounds=(0, 10),
             initialize=stream_flow_initial_guess)
for stream, ub in stream_flow_upper_bound_data.items():
    m.flow[stream].setub(ub)

m.min_cost = Objective(
    expr=sum(m.unit_fixed_cost[unit] for unit in m.units) +
    sum(m.flow[stream] * m.stream_variable_cost[stream]
        for stream in m.streams) +
    122,  # objective constant term
    sense=minimize)

@m.Disjunct(m.units)
def use_unit_disj(disj_blk, unit):
    disj_blk.constraints = ConstraintList()
    # Add the fixed cost constraint to each disjunct, since it has a standard form
    disj_blk.constraints.add(m.unit_fixed_cost[unit] == m.fixed_cost[unit])

# Note: in a more structured model, we would make greater use of indexed variables and constraints
unit_io_constraints = {
    1: [exp(m.flow[3]) - 1 == m.flow[2],
        m.flow[4] == 0, m.flow[5] == 0],
    2: [exp(m.flow[5] / 1.2) - 1 == m.flow[4],
        m.flow[2] == 0, m.flow[3] == 0],
    3: [1.5 * m.flow[9] + m.flow[10] == m.flow[8]],
    4: [1.25 * (m.flow[12] + m.flow[14]) == m.flow[13],
        m.flow[15] == 0],
    5: [m.flow[15] == 2 * m.flow[16],
        m.flow[12] == 0, m.flow[14] == 0],
    6: [exp(m.flow[20] / 1.5) - 1 == m.flow[19],
        m.flow[21] == 0, m.flow[22] == 0],
    7: [exp(m.flow[22]) - 1 == m.flow[21],
        m.flow[19] == 0, m.flow[20] == 0],
    8: [exp(m.flow[18]) - 1 == m.flow[10] + m.flow[17],]
}
for unit in m.units:
    for constr in unit_io_constraints[unit]:
        m.use_unit_disj[unit].constraints.add(constr)

m.use_unit_1or2 = Disjunction(expr=[m.use_unit_disj[1], m.use_unit_disj[2]])
m.use_unit_3ornot = Disjunction(expr=[m.use_unit_disj[3], [m.flow[9] == 0, m.flow[10] == m.flow[8]]])
m.use_unit_4or5ornot = Disjunction(expr=[m.use_unit_disj[4], m.use_unit_disj[5], [m.flow[12] == 0, m.flow[14] == 0, m.flow[15] == 0]])
m.use_unit_6or7ornot = Disjunction(expr=[m.use_unit_disj[6], m.use_unit_disj[7], [m.flow[21] == 0, m.flow[22] == 0, m.flow[19] == 0, m.flow[20] == 0]])
m.use_unit_8ornot = Disjunction(expr=[m.use_unit_disj[8], [m.flow[10] == 0, m.flow[17] == 0, m.flow[18] == 0]])

m.use_unit = BooleanVar(m.units)
for unit in m.units:
    # associate our Boolean variables with the auto-generated disjunct binary variables
    m.use_unit[unit].set_binary_var(m.use_unit_disj[unit].indicator_var)
    
m.global_mass_balances = ConstraintList()
m.global_mass_balances.add(m.flow[13] == m.flow[19] + m.flow[21])
m.global_mass_balances.add(m.flow[17] == m.flow[9] + m.flow[16] + m.flow[25])
m.global_mass_balances.add(m.flow[11] == m.flow[12] + m.flow[15])
m.global_mass_balances.add(m.flow[3] + m.flow[5] == m.flow[6] + m.flow[11])
m.global_mass_balances.add(m.flow[6] == m.flow[7] + m.flow[8])
m.global_mass_balances.add(m.flow[23] == m.flow[20] + m.flow[22])
m.global_mass_balances.add(m.flow[23] == m.flow[14] + m.flow[24])

m.process_specs = ConstraintList()
m.process_specs.add(m.flow[10] <= 0.8 * m.flow[17])
m.process_specs.add(m.flow[10] >= 0.4 * m.flow[17])
m.process_specs.add(m.flow[12] <= 5 * m.flow[14])
m.process_specs.add(m.flow[12] >= 2 * m.flow[14])

m.logical_propositions = LogicalStatementList()
m.logical_propositions.add((m.use_unit[1] | m.use_unit[2]).implies(m.use_unit[3] | m.use_unit[4] | m.use_unit[5]))
m.logical_propositions.add(m.use_unit[3].implies(m.use_unit[8]))
m.logical_propositions.add(m.use_unit[4].equivalent_to(m.use_unit[6] | m.use_unit[7]))

# Optimal solution uses units 2, 4, 6, 8 with objective value 68.
# m.pprint()  # uncomment to print all model objects
print("Model built.")

Model built.


In [3]:
TransformationFactory('core.logical_to_linear').apply_to(m)
SolverFactory('gdpopt').solve(m, tee=False, nlp_solver='gams')
# TransformationFactory('gdp.bigm').apply_to(m)
# SolverFactory('gams').solve(m)
update_boolean_vars_from_binary(m)
# m.display()

In [4]:
# m.display()
m.min_cost.display()
m.use_unit.display()

min_cost : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 68.00974405106514
use_unit : Size=8, Index=units
    Key : Value : Fixed : Stale
      1 : False : False :  True
      2 :  True : False :  True
      3 : False : False :  True
      4 :  True : False :  True
      5 : False : False :  True
      6 :  True : False :  True
      7 : False : False :  True
      8 :  True : False :  True
